In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

from tqdm import tqdm

from show_spectra import plot_spectra

In [2]:
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 

In [3]:
DF = pd.read_csv('../data/main_table.csv')
DF = DF.loc[DF.w_median.notnull()]
Q_MEDIAN = np.load('../data/q_median.npy')
DATA_PATH = '../data/spectra'

# is_nonvar = DF.nonvar_match_num == 1
# is_var    = DF.var_match_num == 1
# is_quasar = DF.quasar_match_num == 1

# nonvar_subset = DF[is_nonvar & ~is_var & ~is_quasar]
# var_subset    = DF[~is_nonvar & is_var & ~is_quasar]
# quasar_subset = DF[~is_nonvar & ~is_var & is_quasar]

In [4]:
LIM_MODE_RMS_UPPER = 0.065
LIM_MODE_MED_LOWER = 0.66
LIM_MODE_MED_UPPER = 0.79
LIM_RMS_LOWER = 0.1
LIM_RMS_UPPER = 0.2

In [5]:
def line(p1, p2):
    x1, y1 = p1
    x2, y2 = p2
    a = (y2-y1)/(x2-x1)
    b = y1 - a*x1
    return lambda x: a * x + b

def cc_region(df):
    line_lower = line((0.7, 0.1), (3.7, 1.5))
    line_upper = line((0.7, 0.5), (2.0, 1.1))
    x = df['psfMag_u'] - df['psfMag_g']
    y = df['psfMag_g'] - df['psfMag_r']
    
    lux, llx = line_upper(x), line_lower(x)
    
    conditions = [
        (x < 0.7) & (y < -0.2),
        (x < 0.7) & (-0.2 <= y) & (y <= 0.5),
        (x < 0.7) & (y > 0.5),
        (x < 2.0) & (y > lux),
        (0.7 <= x) & (x <= 1.4) & (y < llx),
        (0.7 <= x) & (x <= 1.4) & (y > llx) & (y < lux),
        (1.4 < x) & (x < 2.0) & (y > llx) & (y < lux),
        (x >= 2.0) & (y > llx)        
    ]
    choices = [ 1, 2, 3, 3, 4, 5, 6, 7 ]
    
    return np.select(conditions, choices, default=8)

def ww_region(df):
    x = df['w_rrms']
    y = df['w_median']
    is_nonvar = df.nonvar_match_num == 1
    is_var    = df.var_match_num == 1
    is_quasar = df.quasar_match_num == 1
    
    conditions = [
        is_nonvar & ~is_var & ~is_quasar & (x > 0.1) & (x < 0.2) & (y < 0.3),
        is_nonvar & ~is_var & ~is_quasar & (x < 0.03) & (y > 0.7) & (y < 0.8),
        is_nonvar & ~is_var & ~is_quasar & (x > 0.15),
        ~is_nonvar & is_var & ~is_quasar & (x > 0.10),
        ~is_nonvar & ~is_var & is_quasar & (x > 0.10),
        is_nonvar & ~is_var & ~is_quasar & (x < 0.02) & (y > 0.99) & (y < 1.01),
    ]
    choices = [ 4, 5, 1, 2, 3, 6 ]
    
    return np.select(conditions, choices, default=7)

def ww_region_rms(df):
    x = df['w_rrms']
    y = df['w_median']
    
    conditions = [
        (x < LIM_MODE_RMS_UPPER) & (y < LIM_MODE_MED_UPPER) & (y > LIM_MODE_MED_LOWER),
        (x < LIM_RMS_UPPER) & (x > LIM_RMS_LOWER),
        (x > LIM_RMS_UPPER)
    ]
    choices = [ 1, 2, 3 ]
    
    return np.select(conditions, choices, default=7)
    

cc_region_names = [ f'R_CC_{n}' for n in [ 'I', 'II', 'III', 'IV', 'Va', 'Vb', 'Vc', 'VI' ] ]
ww_region_names = [ f'R_WW_{n}' for n in '1234567' ]
ww_region_rms_names = [ f'R_WW_{n}' for n in '1237' ]

# new

In [6]:
# DF['wwr'] = ww_region(DF)
DF['wwr'] = ww_region_rms(DF)
DF['ccr'] = cc_region(DF)

In [7]:
n = 0

In [8]:
# for gid, wwr, ccr in tqdm(DF[DF.wwr != 7][['source_id', 'wwr', 'ccr']].to_numpy()[n*1000:(n+1)*1000]):
for gid, wwr, ccr in tqdm(DF[DF.wwr != 7][['source_id', 'wwr', 'ccr']].to_numpy()):
    path = f'../figures/roi/{ww_region_names[wwr-1]}/{cc_region_names[ccr-1]}/'
    if not os.path.exists(path):
        os.makedirs(path)
        
    plot_spectra(gaia_id=gid, savefig=path, mt=DF, q_med=Q_MEDIAN, data_path=DATA_PATH)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12953/12953 [1:29:30<00:00,  2.41it/s]


In [7]:
uncl = DF[~(is_nonvar | is_var | is_quasar)]

In [8]:
uncl[(uncl.wwr == 7) & (uncl.w_median > 0.7) & (uncl.w_median < 0.8) & (uncl.w_rrms < 0.05)]

,source_id,clean_sdssdr13_oid,original_ext_source_id,angular_distance,gaia_ra,gaia_dec,phot_g_mean_mag,bp_g,g_rp,objId,...,w_mean,w_std,w_median,w_rrms,D_mean,D_std,D_median,D_rrms,wwr,ccr
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
3,2686850112096922368,63997432,1237663456705511599,0.071691,323.518261,-0.691753,16.003223,0.421000,0.581319,1237663456705511599,...,0.761893,0.025566,0.757452,0.013946,0.238107,0.025566,0.242548,0.013946,7,6
74,2686622375747773568,58139427,1237656567043326051,0.054710,323.888410,-1.192175,15.584722,0.431451,0.608807,1237656567043326051,...,0.728769,0.030536,0.720564,0.035514,0.271231,0.030536,0.279436,0.035514,7,6
114,2686838292346697472,60682541,1237657189815353635,0.060470,323.229283,-0.970831,16.814514,0.428482,0.591326,1237657189815353635,...,0.735515,0.022913,0.729443,0.027370,0.264485,0.022913,0.270557,0.027370,7,6
159,2686837845670143488,60686429,1237666185626255558,0.120809,323.295130,-0.970518,14.655992,0.384148,0.557196,1237666185626255558,...,0.723543,0.021367,0.721457,0.015522,0.276457,0.021367,0.278543,0.015522,7,4
371,2686847844354711296,63049403,1237663478180741184,0.063926,323.440715,-0.769323,14.601306,0.400590,0.579837,1237663478180741184,...,0.745234,0.024285,0.739799,0.020740,0.254766,0.024285,0.260201,0.020740,7,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21955,2688237347874618880,82895778,1237678617402212600,0.026591,324.342030,0.846337,17.633123,0.309515,0.479462,1237678617402212600,...,0.746271,0.026033,0.753868,0.021805,0.253729,0.026033,0.246132,0.021805,7,5
21972,2646228655005377792,87195647,1237678617415057435,0.155451,353.609378,1.214047,17.219310,0.242834,0.419060,1237678617415057435,...,0.733811,0.053346,0.716123,0.015630,0.266189,0.053346,0.283877,0.015630,7,5
22054,2688216598888416768,84185220,1237663458852864528,0.048702,323.279361,0.956930,16.549961,0.533588,0.683401,1237663458852864528,...,0.744125,0.070217,0.734538,0.016859,0.263323,0.032609,0.265766,0.016438,7,7


In [ ]:
for gid, w_median, w_rrms, ccr in tqdm(uncl[['source_id', 'w_median', 'w_rrms', 'ccr']].to_numpy()):
    if 0.7 < w_median < 0.8 and w_rrms < 0.05:
        path = f'../figures/roi/ambi5/{cc_region_names[int(ccr-1)]}/'
    else:
        continue
        
    if not os.path.exists(path):
        os.makedirs(path)
        
    plot_spectra(gaia_id=gid, savefig=path, mt=DF, q_med=Q_MEDIAN, data_path=DATA_PATH)

  0%|                                                                                        | 3/5760 [00:00<08:43, 11.00it/s]

500 Error 500:
No handler found for this request.

Cannot process request: 'https://gea.esac.esa.int/data-server/data' (req: Reqid: anonymous1707592583943, retrieval access: DIRECT, retrieval type: XP_CONTINUOUS, compression: null), for user: UwsJobOwner{id='anonymous', name='null', mail='null', authUsername='null', authGroups=[], pseudo='anonymous', session='6E6A1206B6CEBAC60F427476EB00ED1D', ip='88.207.16.122', roles=0, parameters=Owner parameters: 6},  due to: No handler found for this request.


  0%|▎                                                                                      | 20/5760 [00:04<21:25,  4.47it/s]

500 Error 500:
No handler found for this request.

Cannot process request: 'https://gea.esac.esa.int/data-server/data' (req: Reqid: anonymous1707592588029, retrieval access: DIRECT, retrieval type: XP_CONTINUOUS, compression: null), for user: UwsJobOwner{id='anonymous', name='null', mail='null', authUsername='null', authGroups=[], pseudo='anonymous', session='CACF9FAAAFCCB7CF321716FF48033672', ip='88.207.16.122', roles=0, parameters=Owner parameters: 6},  due to: No handler found for this request.


  0%|▍                                                                                      | 28/5760 [00:08<31:56,  2.99it/s]

...

Could not get SDSS data
Could not get SDSS data


  6%|█████▏                                                                               | 355/5760 [00:24<00:48, 111.54it/s]

Could not get SDSS data
Could not get SDSS data
500 Error 500:
No handler found for this request.

Cannot process request: 'https://gea.esac.esa.int/data-server/data' (req: Reqid: anonymous1707592612232, retrieval access: DIRECT, retrieval type: XP_CONTINUOUS, compression: null), for user: UwsJobOwner{id='anonymous', name='null', mail='null', authUsername='null', authGroups=[], pseudo='anonymous', session='844080F772285FDB87F25F13A8249FBD', ip='88.207.16.122', roles=0, parameters=Owner parameters: 6},  due to: No handler found for this request.
Could not get SDSS data


  6%|█████▌                                                                                | 371/5760 [00:33<11:33,  7.77it/s]

500 Error 500:
No handler found for this request.

Cannot process request: 'https://gea.esac.esa.int/data-server/data' (req: Reqid: anonymous1707592617219, retrieval access: DIRECT, retrieval type: XP_CONTINUOUS, compression: null), for user: UwsJobOwner{id='anonymous', name='null', mail='null', authUsername='null', authGroups=[], pseudo='anonymous', session='237E4D6C4835EBF5573F52E00B1D06B7', ip='88.207.16.122', roles=0, parameters=Owner parameters: 6},  due to: No handler found for this request.
